In [1]:
import pandas as pd

### Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

-если источник traffic_source равен yandex или google, то в source_type ставится organic;
-для источников paid и email из России - ставим ad;
-для источников paid и email не из России - ставим other;
-все остальные варианты берем из traffic_source без изменений

In [2]:
#данные
log = pd.read_csv('p3_data/visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [3]:
#функция
def source(row):
    if (row['traffic_source']=='yandex') | (row['traffic_source']=='google'):
        return 'organic'
    elif ((row['traffic_source']=='paid') | (row['traffic_source']=='email')) & (row['region'] == 'Russia'):     
        return 'ad'
    elif ((row['traffic_source']=='paid') | (row['traffic_source']=='email')) & (row['region'] != 'Russia'):     
        return 'other'        
    else: 
        return row['traffic_source']

In [4]:
#новый столбцец
log['source_type'] = log.apply(source, axis=1)
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [5]:
url = pd.read_csv('p3_data/urls.txt')
url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [6]:
url[url.url.str.contains(r'/\d{8}-', regex=True)].head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


### Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

Ratings Data File Structure (ratings.csv)
-----------------------------------------

All ratings are contained in the file `ratings.csv`. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

    userId,movieId,rating,timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

In [8]:
# данные
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

# считаю для каждого юзера кол-во оценок, мин и макс даты (по описанию датасета у одного юзера фильм не будет повторяться, так что ок)
all_users = ratings[['userId', 'timestamp']].groupby('userId').agg(['count','min','max']).reset_index()
# добавляю столбец с разницей макс и мин.дат
all_users['lifetime_in_sec'] = all_users['timestamp']['max'] - all_users['timestamp']['min']
all_users.head()

# выделяю пользователей, которые выставили более 100 оценок
active_users = all_users[['userId', 'lifetime_in_sec']][all_users['timestamp']['count'] > 100]
active_users.head()

# средняя продолжительность жизни у активных пользователей в секундах
result_in_sec = active_users.lifetime_in_sec.mean()
print(result_in_sec)

# средняя продолжительность жизни у активных пользователей в днях
result_in_days = result_in_sec / 60 / 60 / 24
print(result_in_days)

# средняя продолжительность жизни у активных пользователей в месяцах
result_in_months = result_in_days / 30
print(result_in_months)


40080507.4496124
463.89476214829165
15.463158738276388


### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [10]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
# таблица с тремя типами выручки для каждого client_id без указания адреса клиента
all_revenue = rzd.merge(air, how = 'outer', on = 'client_id').merge(auto, how = 'outer', on = 'client_id')
all_revenue

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,57483.0
3,114,5774.0,NaN,83.0
4,115,981.0,81.0,912.0
5,116,NaN,4.0,4834.0
6,117,NaN,13.0,98.0
7,118,NaN,173.0,NaN


In [14]:
# с указанием адреса клиента (left, тк нас вряд ли интересуют клиенты без выручки, если бы такие были)
with_address = all_revenue.merge(client_base, how = 'left', on = 'client_id')
with_address

,client_id,rzd_revenue,air_revenue,auto_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,NaN,57483.0,Левобережная 1а
3,114,5774.0,NaN,83.0,Мира 14
4,115,981.0,81.0,912.0,ЗЖБИиДК 1
5,116,NaN,4.0,4834.0,Строителей 18
6,117,NaN,13.0,98.0,Панфиловская 33
7,118,NaN,173.0,NaN,Мастеркова 4
